In [1]:
import pandas as pd
import numpy as np
import itertools as it
import matplotlib.pyplot as plt
from scipy import stats
from collections import defaultdict
import csv

In [18]:
players = pd.read_csv('./data/players.csv')
games = pd.read_csv('./data/games.csv')
plays = pd.read_csv('./data/plays.csv')
tackle_scores = pd.read_csv('./intermediate_data/tackle_acc_and_totals.csv')
AVG_TACKLE_SCORE = np.average(tackle_scores.score)

In [11]:
def getClosest(players, football):
    distances = []
    for index, row in players.iterrows():
        distance = np.linalg.norm(np.subtract([football.x.values[0], football.y.values[0]], [row.x, row.y]))
        distances.append((row.nflId, distance))
    closest = sorted(distances, key= lambda t: t[1])[0][0]
    return closest

In [23]:
play_tackler_scores = defaultdict(dict)
for week_num in range(1, 18):
    print(week_num)
    g_week = games[games['week'] == week_num]
    week_str = './data/week' + str(week_num) + '.csv'
    current_week_data = pd.read_csv(week_str)
    
    for _, g_row in g_week.iterrows():
        GAMEID = g_row.gameId
        cwg_data = current_week_data[current_week_data['gameId'] == GAMEID]
        pg = plays[plays['gameId'] == GAMEID]
        
        for _, p_row in pg.iterrows():
            
            if p_row.passResult == 'C':
                PLAYID = p_row.playId
                curr_play_data = cwg_data[cwg_data['playId'] == PLAYID]
                
                play_events = pd.unique(curr_play_data.event)                     
                if 'pass_outcome_caught' in play_events:
                    try:
                        OFFENSIVE_TEAM = curr_play_data[curr_play_data['position'] == 'QB'].team.values[0]
                        DEFENSIVE_TEAM = ('home' if OFFENSIVE_TEAM == 'away' else 'away')
                    except:
                        continue
                    
                    pc_data = curr_play_data[curr_play_data['event'] == 'pass_outcome_caught']
                    football1 = pc_data[pc_data['team'] == 'football']
                    offensive_players = pc_data[pc_data['team'] == OFFENSIVE_TEAM]
                    defensive_players = pc_data[pc_data['team'] == DEFENSIVE_TEAM]
                    try:
                        closest1 = getClosest(offensive_players, football1)
                    except:
                        continue
                        
                    receiver = pc_data[pc_data['nflId'] == closest1]
                    rec_x = receiver.x.values[0]
                    rec_y = receiver.y.values[0]

                    defendersCloseCaught = {}
                    for _, row in defensive_players.iterrows():
                        def_x = row.x
                        def_y = row.y
                        dist = np.linalg.norm(np.subtract([rec_x, rec_y], [def_x, def_y]))
                        if dist <= 10:
                            defendersCloseCaught[row.nflId] = dist
                    
                    tackler_score = 0
#                     print(defendersCloseCaught)
                    for nflId, dist in defendersCloseCaught.items():
                        if nflId in list(tackle_scores.nflId):
                            t_sc = tackle_scores[tackle_scores['nflId'] == nflId].score.values[0]
                        else:
                            t_sc = AVG_TACKLE_SCORE
                        tackler_score += (1 / dist) * (t_sc)
                    
                    play_tackler_scores[GAMEID][PLAYID] = tackler_score

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [24]:
with open('./intermediate_data/tacklers_per_play.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(['gameId', 'playId', 'tacklers_agg'])
    for GAME in play_tackler_scores:
        for PLAY, val in play_tackler_scores[GAME].items():
            writer.writerow([GAME, PLAY, val])